In [ ]:
from keras.models import load_model
import numpy as np
import sys
import os
import pickle as pk
import pandas as pd
from os import listdir
from os.path import isfile, join

from keras.utils import to_categorical ,Sequence
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras.layers import (Convolution1D, Dense, Dropout, GlobalAveragePooling1D, 
                          GlobalMaxPool1D, Input, MaxPool1D, concatenate)
from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation)
from keras import backend as K                      

# calculate accuracy
from sklearn.metrics import accuracy_score
pd.options.mode.chained_assignment = None

In [ ]:
map_dict = pk.load(open('data/map.pkl' , 'rb'))
reverse_dict = pk.load(open('data/map_reverse.pkl' , 'rb'))

name = pd.read_csv('data/sample_submission.csv')
name = name['fname'].tolist()

# Parameter

In [ ]:
model_path = 'model_full_resnet34'
predict_path = 'model_full_resnet34_csv'

unverified_path = 'jerry_resnet34_mfcc3_unverified_{}.csv'
test_path = 'jerry_resnet34_mfcc3_test_{}.csv'

cotrain_path = join((predict_path),'jerry_mfcc3_resnet34_mixup_cotrain_Y.csv')

weight = [0.86253 ,0.85445 ,0.84636 ,0.87601 ,0.8814 ,0.86523 ,0.88949 ,0.85714 ,0.87871 ,0.85714]

if not os.path.exists(predict_path):
    os.mkdir(predict_path)

# Predict all validation  for each fold

In [ ]:

    
for i in range(1,11):
    print('fold {}'.format(i))
    print('#'*50)
    
    model = load_model(join(model_path,'best_{}.h5'.format(i)))
    
    df_list = []

    for j in range(1,11):
        val_name = np.load("data/ten_fold_data/valid_fname_{}.npy".format(j))
        val_predict = model.predict(np.load('data/ten_fold_data/X_valid_{}.npy'.format(j)) ,  verbose=1)
        df = pd.DataFrame(val_predict)
        df.insert(0,'fname' , val_name)
        df_list.append(df)
    
    df = pd.concat(df_list)
    df.to_csv(join(predict_path, 'fold_{}.csv'.format(i)),index=False)
    
    
        

# Predict Unverified data && Testing data

In [ ]:

audio_martix = np.load('data/train/mfcc3/X_train.npy')

df = pd.read_csv('data/train_label.csv')  
df_unmanu = df[df['manually_verified'] == 0]
df_unmanu['trans'] = df_unmanu['label'].map(map_dict)
df_unmanu['onehot'] = df_unmanu['trans'].apply(lambda x: to_categorical(x,num_classes=41))
unmanu_veri_idx = df_unmanu.index.values
un_fnames = df_unmanu['fname'].values

X = audio_martix[unmanu_veri_idx]
Y = df_unmanu['onehot'].tolist()
Y = np.array(Y)
Y = Y.reshape(-1 ,41)

test_X = np.load("data/test/mfcc3/X_test.npy")
test_fnames = pd.read_csv('data/sample_submission.csv')['fname'].values


# predict unverified data and testing data
for i in range(1,11):
    model = load_model(join(model_path,'best_{}.h5'.format(i)))
    
    unverified_predict = model.predict(X ,  verbose=1 , batch_size = 32)
    unver_df = pd.DataFrame(unverified_predict)
    unver_df.insert(0,'fname' , un_fnames)
    unver_df.to_csv(join(predict_path, unverified_path.format(i)),index=False) 
    
    test_predict = model.predict(test_X ,  verbose=1 , batch_size = 32)
    test_df = pd.DataFrame(test_predict)
    test_df.insert(0,'fname' , test_fnames)
    test_df.to_csv(join(predict_path, test_path.format(i)),index=False) 

    #concat_df = pd.concat([unver_df,test_df])
    #concat_df.to_csv(join(predict_path, 'concat_{}.csv'.format(i)),index=False) 


# Ensemble 10 Fold  with weignt 

In [ ]:

result = 0.0
fname = []

for i in range(1,11): 
    
    test_df = pd.read_csv(join(predict_path, test_path.format(i)))
    uverified_df = pd.read_csv(join(predict_path,unverified_path.format(i)))
    
    concat_df = pd.concat([uverified_df,test_df])
    fname = concat_df['fname'].values
    
    col_name = [str(x)for x in list(range(0,41))]
    predict = concat_df[col_name].values
    
    result += predict*weight[i-1] / sum(weight)

print('weight ensemble calculate done')
max_confidence = np.amax(result, axis=1) 

mean_confidence = np.mean(max_confidence)
std_confidence = np.std(max_confidence)

threshold = mean_confidence + std_confidence

semi_idx = []
for idx,v in enumerate(result):
    if np.max(v) > threshold:
        semi_idx.append(idx)

# Filter data chich probabilistic > threshold

In [ ]:
semi_result = result[semi_idx]
semi_result = [np.argmax(x) for x in semi_result]
semi_fname = fname[semi_idx]

# print(semi_result.shape)
# print(semi_fname.shape)

cotrain_df = pd.DataFrame({'fname':semi_fname , 'label_verified':semi_result})
cotrain_df.to_csv(cotrain_path,index=False) 

semi_result[0:3]